In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import random
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis, 
                                            QuadraticDiscriminantAnalysis)
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
import pickle

In [2]:
ds = pd.read_csv('./musk.data', header = None)
# Quitamos las dos primeras columnas ya que son nombres simbólicos
ds = ds.drop([0,1], axis=1)

ds.columns -= 2

output = ds.shape[1]-1 # Columna de output
y = ds[output].values
X = ds.drop(output,axis=1).values
seed = 42
random.seed(seed)
np.random.seed(seed)

In [3]:
X = (X - np.mean(X)) / np.std(X)
X,y = shuffle(X,y)

In [4]:
def principal_component_analisys(X_train, X_test, th):
    th = .85
    alg = PCA()
    X_train = alg.fit_transform(X_train)
    X_test = alg.transform(X_test)
    suma = 0
    var = alg.explained_variance_ratio_
    for i,j in enumerate(var):
        suma += j
        if suma >= th:
            X_train, X_test = (X_train[:,:i + 1], X_test[:,:i + 1])
            break
    return X_train, X_test
        

In [5]:
def exec_MLPC():
    global resultados_train
    global resultados_test
    global res_train
    global res_test
    global model
    global train_indices
    global test_indices
    global etiquetas
    global th
    etiquetas.extend(['MLPR sgd 10 relu',
                 'MLPR sgd 30 relu',
                 'MLPR sgd 50 relu',
                 'MLPR adam 10 relu',
                 'MLPR adam 30 relu',
                 'MLPR adam 50 relu'])
    etiquetas.extend(['MLPR sgd 10 tanh',
                 'MLPR sgd 30 tanh',
                 'MLPR sgd 50 tanh',
                 'MLPR adam 10 tanh',
                 'MLPR adam 30 tanh',
                 'MLPR adam 50 tanh'])
    dicc = [{'hidden_layer_sizes': (10,), 'solver': 'sgd',  
             'activation': 'relu','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (30,), 'solver': 'sgd',  
             'activation': 'relu','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (50,), 'solver': 'sgd',  
             'activation': 'relu','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (10,), 'solver': 'adam', 
             'activation': 'relu','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (30,), 'solver': 'adam', 
             'activation': 'relu','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (50,), 'solver': 'adam', 
             'activation': 'relu','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (10,), 'solver': 'sgd',  
             'activation': 'tanh','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (30,), 'solver': 'sgd',  
             'activation': 'tanh','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (50,), 'solver': 'sgd',  
             'activation': 'tanh','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (10,), 'solver': 'adam', 
             'activation': 'tanh','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (30,), 'solver': 'adam', 
             'activation': 'tanh','early_stopping': True, 'max_iter': 1500},
            {'hidden_layer_sizes': (50,), 'solver': 'adam', 
             'activation': 'tanh','early_stopping': True, 'max_iter': 1500}]

    for i in range(len(dicc)):
        train_indices2 = []
        test_indices2 = []
        for j in range(3):
            for train_index, test_index in kf.split(X):
                X_train, X_test = X[train_index],X[test_index]
                y_train, y_test = y[train_index],y[test_index]
                X_train_pca, X_test_pca = principal_component_analisys(X_train, X_test, th)
                train_indices2.append(train_index)
                test_indices2.append(test_index)
                alg = MLPClassifier(**dicc[i])
                alg.fit(X_train_pca, y_train)
                model.append(alg)
                resultados_train.append(alg.score(X_train_pca, y_train))
                resultados_test.append(alg.score(X_test_pca, y_test))
        res_train = np.hstack((res_train, np.array(resultados_train,ndmin=2).T))
        res_test = np.hstack((res_test, np.array(resultados_test,ndmin=2).T))
        resultados_train = []
        resultados_test = []
        train_indices.append(train_indices2)
        test_indices.append(test_indices2)


In [6]:
%%time
kf = KFold(n_splits=10,shuffle=True)
resultados_train = []
resultados_test = []
model = []
res_train = np.empty((30,0))
res_test = np.empty((30,0))
train_indices = []
test_indices = []
etiquetas = []
th = 0.85
exec_MLPC()

/home/diego/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/diego/.local/lib/python3.6/site-packages/IPython/core/magics/execution.py", line 1292, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 11, in <module>
  File "<ipython-input-5-3872acd434bd>", line 55, in exec_MLPC
    X_train_pca, X_test_pca = principal_component_analisys(X_train, X_test, th)
  File "<ipython-input-4-2b250d8a8a87>", line 4, in principal_component_analisys
    X_train = alg.fit_transform(X_train)
  File "/home/diego/.local/lib/python3.6/site-packages/sklearn/decomposition/pca.py", line 359, in fit_transform
    U, S, V = self._fit(X)
  File "/home/diego/.local/lib/python3.6/site-packages/sklearn/decomposition/pca.py", line 406, in _fit
    return self._fit_full(X, n_components)
  File "/home/diego/.local/lib/python3.6/site-packages/sklearn/decomposition/pca.py", line 437, in _fit_full
    U, S, V = linalg.svd(X, full_matrices=False)
  File "/home/diego/.local/lib/python3.6/site-packages/scipy/linalg/dec

KeyboardInterrupt: 

In [7]:
with open('p5_musk-res_test.bin','wb') as file:
    pickle.dump(res_test, file)
with open('p5_musk-train_indices.bin','wb') as file:
    pickle.dump(train_indices, file)
with open('p5_musk-test_indices.bin','wb') as file:
    pickle.dump(test_indices, file)
with open('p5_musk-models.bin','wb') as file:
    pickle.dump(model, file)
with open('p5_musk-etiquetas.bin','wb') as file:
    pickle.dump(etiquetas, file)

In [8]:
with open('p5-res_test.bin','rb') as file:
    res_test = pickle.load(file)
with open('p5-train_indices.bin','rb') as file:
    train_indices = pickle.load(file)
with open('p5-test_indices.bin','rb') as file:
    test_indices = pickle.load(file)
with open('p5-models.bin','rb') as file:
    model = pickle.load(file)
with open('p5-etiquetas.bin','rb') as file:
    etiquetas = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'p5-res_test.bin'

In [ ]:
np.apply_along_axis(lambda x: stats.shapiro(x)[1], axis=0, arr=res_test)

In [ ]:
print(stats.kruskal(*zip(*list(res_test))))

In [ ]:
labels = [etiquetas[i] for i in range(res_test.shape[1]) for _ in range(res_test.shape[0])]
arr_test = res_test.flatten('F')
resultados = pairwise_tukeyhsd(arr_test,labels)
# resultados.summary()

In [ ]:
#seleccionamos el que tiene mejor media y vemos en el anterior cuales son iguales
best_group = np.argmax(np.mean(res_test, axis = 0))
best_group = etiquetas[int(best_group)]

In [ ]:
print(resultados.plot_simultaneous(comparison_name=best_group))

In [ ]:
resultados = resultados._results_table.data

In [ ]:
group1 = resultados[0].index('group1')
group2 = resultados[0].index('group2')
reject = resultados[0].index('reject')
best_groups = []
for row in resultados[1:]:
    if (row[group1] == str(best_group) or row[group2] == str(best_group)) and not row[reject]:
        best_groups.append(row[group1])
        best_groups.append(row[group2])
best_groups.append(best_group)
best_groups = set(best_groups)
sorted(best_groups)

In [ ]:
def performance_indexes(CM, PositiveClass):
    classNum = len(CM)
    TP = CM[PositiveClass, PositiveClass]
    TN = 0
    FP = 0
    FN = 0
    for real in range(classNum):
        for predicted in range(classNum):
            if (real != PositiveClass and predicted != PositiveClass):
                TN += CM[real, predicted]
            if (real != PositiveClass and predicted == PositiveClass):
                FP += CM[real, predicted]
            if (real == PositiveClass and predicted != PositiveClass):
                FN += CM[real, predicted]
    Sens = TP/(TP+FN)
    Spec = TN/(TN+FP)
    PPV = TP/(TP+FP)
    NPV = TN/(TN+FN)
    ACC = (TP+TN)/(TP+TN+FP+FN)
    return Sens,Spec,PPV,NPV,ACC

In [ ]:
for etiqueta in sorted(best_groups):
    i = etiquetas.index(etiqueta)
    m_min = i*30
    m_max = (i+1)*30
    current_model = model[m_min:m_max]
    Sens_l = []
    Spec_l = []
    PPV_l = []
    NPV_l = []
    ACC_l = []
    for idx, test_index in enumerate(test_indices[i]):
        X_test, y_test = X[test_index], y[test_index]
        y_pred = current_model[idx].predict(X_test)
        CM = confusion_matrix(y_test, y_pred)
        Sens,Spec,PPV,NPV,ACC = performance_indexes(CM,1)
        Sens_l.append(Sens)
        Spec_l.append(Spec)
        PPV_l.append(PPV)
        NPV_l.append(NPV)
        ACC_l.append(ACC)
    print(etiqueta,('-> Sens: {:.2f}, Spec: {:.2f}, ' +
           'PPV: {:.2f}, NPV: {:.2f}, ACC: {:.2f}').format(np.mean(Sens_l),
                                                           np.mean(Spec_l),
                                                           np.mean(PPV_l),
                                                           np.mean(NPV_l),
                                                           np.mean(ACC_l)))